In [16]:
import pandas as pd
from joblib import dump
import sklearn.feature_extraction as fe
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

import nltk
from nltk.stem import WordNetLemmatizer

In [17]:
base_dir = 'D:/Users/Vladislav/Documents/Jupyter Notebook/Лабораторные работы/Программная инженерия'

train_data = pd.read_csv(base_dir + '/nlp-getting-started/train.csv')
test_data = pd.read_csv(base_dir + '/nlp-getting-started/test.csv')

stop_words = nltk.corpus.stopwords.words('english')

In [18]:
# used lemms

lemmatizer = WordNetLemmatizer()

lemm_train_data = [lemmatizer.lemmatize(word) for word in train_data['text']]
lemm_test_data = [lemmatizer.lemmatize(word) for word in test_data['text']]

In [19]:
# make tokens

count_vec = fe.text.CountVectorizer(stop_words=stop_words)

#train_tok = count_vec.fit_transform(lemm_train_data)
#test_tok = count_vec.transform(lemm_test_data)


# used TFIDF

tfidf_transformer = fe.text.TfidfTransformer()

#tf_train_tok = tfidf_transformer.fit_transform(train_tok)
#tf_test_tok = tfidf_transformer.transform(test_tok)

In [20]:
# make pipline

pipe = Pipeline( [('count_vect', count_vec), ('tfidf_transf', tfidf_transformer)] )
pipe.fit(lemm_train_data)

dump(pipe, 'pipe_data_transf.joblib')

['pipe_data_transf.joblib']

In [10]:
# transform data

tf_train_tok = pipe.transform(lemm_train_data)
tf_test_tok = pipe.transform(lemm_test_data)

print(tf_train_tok.shape)

(7613, 21498)


In [11]:
clf = MultinomialNB().fit(tf_train_tok, train_data['target'])

In [12]:
predicted = clf.predict(tf_test_tok)

sample_submission = pd.read_csv(base_dir + "/nlp-getting-started/sample_submission.csv")

In [13]:
sample_submission["target"] = predicted

In [14]:
sample_submission.to_csv(base_dir + "/submission.csv", index=False)